# Building your first AzureML web service using Python 

This tutorial demonstrates how to deploy a machine learning model built using Python and scikit-learn. Before running the tutorial, your DSVM must be configured as specified in the README in the [Machine Learning Operationalization](https://aka.ms/o16ncli) GitHub repo.

The tutorial uses the digits dataset that is part of the [scikit-learn distribution](http://scikit-learn.org/stable/tutorial/basic/tutorial.html).

In the tutorial, you will load a dataset, train a model on the dataset, and then publish a
realtime scoring API for the model.

To complete this tutorial:

* Sign in to your DSVM.
* Copy this notebook to a folder on the DSVM.
* In a browser, open https://&lt;your dsvm ip address&gt;:&lt;port&gt; and sign into the Jupyter server.
    * On a Linux DSVM the port is typically 8000
    * On a Windows DSVM the port is typically 9999
* In Jupyter, navigate to the folder and open the notebook.


In [ ]:
# Read in the digits dataset
from sklearn import datasets
digits = datasets.load_digits()
print(digits.data)

## Train your model

The task is to predict which digit an image represents in the digits dataset. There are samples of each of the 10 possible classes (the digits zero through nine) on which you *fit* an estimator to predict the classes to which unseen samples belong.

In [ ]:
# Train an SVM classifier
from sklearn import svm
clf = svm.SVC(gamma=0.001, C=100.)
clf.fit(digits.data[:-1], digits.target[:-1])

In [ ]:
# Predict new inputs
clf.predict(digits.data[-1:])[0]

### Save your model

Once you have a model that performs well, you can package it into a scoring service. To prepare for this, save your model locally. You then use the Joblib library to pickle the model.

In [ ]:
# Save model
from sklearn.externals import joblib
import os
if not os.path.exists('sklearn'):
    os.makedirs('sklearn')
joblib.dump(clf, 'sklearn/model.pkl')
print("Model saved")

## Authoring a Realtime Web Service

In this section, you author a realtime web service that scores the model you saved above. 

### Define ```init``` and ```run```

Start by defining your ```init``` and ```run``` functions in the cell below. 

The ```init``` function initializes the web service, loading in any data or models that it needs to score your inputs. In the example below, it loads in the trained model and the schema of your dataset.

The ```run``` function defines what is executed on a scoring call. In this simple example, the service loads the json input as a data frame and runs the pipeline on the input.

In [ ]:
#%%save_file -f skdigits.py
# Prepare the web service definition by authoring
# init() and run() functions. Once tested, remove
# the commented magic on the first line to save
# the cell to a file.
def init():
    # install sklearn if not already present
    try:
        import scipy
        from sklearn.externals import joblib
        import numpy
    except ImportError:
        import pip
        pip_args = ['install', 'sklearn', 'scipy']
        pip.main(pip_args)
        
        import scipy
        from sklearn.externals import joblib
        import numpy
        
    # read in the model file
    from sklearn.externals import joblib
    global model
    model = joblib.load('sklearn/model.pkl')
        
def run(inputString):
    import json
    import numpy
    try:
        input_list=json.loads(inputString)
    except ValueError:
        return 'Bad input: expecting a json encoded list of lists.'
    
    input_array = numpy.array(input_list)
    if (input_array.shape != (1, 64)):
        return 'Bad input: Expecting a json encoded list of lists of shape (1,64).'
    else:
        pred = model.predict(input_array)[0]
        return json.dumps(str(pred))



### Test ```init``` and ```run```

Before publishing the web service, you can test the init and run functions in the notebook by running the the following cell.

In [ ]:
import json
init()
run(json.dumps(digits.data[-1:].tolist()))

### Create a script that defines the web service

Your goal is to create an endpoint that you can call to make predictions based on the input data. To create a web service using the model you saved, you start by authoring a script to do the scoring.
 
In the script you identify the input parameters you want your web service to consume and the outputs it should produce. 

Go back to the cell where you defined your ```init``` and ```run``` functions, uncomment the magic in the first line (```#%%save_file -f skdigits.py```), and run the cell again. This saves the contents of the cell to a local file with the name supplied to the ```-f``` argument.


### Use the Azure Machine Learning CLI to deploy and manage your web services

Sign into the DSVM and run the following commands to deploy your service locally.

Before you can run the Azure Machine Learning CLI command to operationalize the model, you must have set the environment variables that you generated when you setup your DSVM. If you did not do so when you configured the DSVM, see the README in the [Machine Learning Operationalization](https://aka.ms/o16ncli) GitHub repo for more information.

Open the folder containing the notebook.

Next run the following commands to create the web service:

```
az ml env local
az ml service create realtime -f skdigits.py -d sklearn -n skdigitsapp -r scikit-py
```

To create and run the web service on the ACS cluster, change to the cluster mode and rerun the service creation command:

```
az ml env cluster
az ml service create realtime -f skdigits.py -d sklearn -n skdigitsapp -r scikit-py
```

To test the local web service, run the following command with a sample data input:

**Linux** 

```
az ml service run realtime -n skdigitsapp -d '{"input":"[[0.0,0.0,10.0,14.0,8.0,1.0,0.0,0.0,0.0,2.0,16.0,14.0,6.0,1.0,0.0,0.0,0.0,0.0,15.0,15.0,8.0,15.0,0.0,0.0,0.0,0.0,5.0,16.0,16.0,10.0,0.0,0.0,0.0,0.0,12.0,15.0,15.0,12.0,0.0,0.0,0.0,4.0,16.0,6.0,4.0,16.0,6.0,0.0,0.0,8.0,16.0,10.0,8.0,16.0,8.0,0.0,0.0,1.0,8.0,12.0,14.0,12.0,1.0,0.0]]"}'
```

**Windows** 

```
az ml service run realtime -n skdigitsapp -d "{\"input\":\"[[0.0,0.0,10.0,14.0,8.0,1.0,0.0,0.0,0.0,2.0,16.0,14.0,6.0,1.0,0.0,0.0,0.0,0.0,15.0,15.0,8.0,15.0,0.0,0.0,0.0,0.0,5.0,16.0,16.0,10.0,0.0,0.0,0.0,0.0,12.0,15.0,15.0,12.0,0.0,0.0,0.0,4.0,16.0,6.0,4.0,16.0,6.0,0.0,0.0,8.0,16.0,10.0,8.0,16.0,8.0,0.0,0.0,1.0,8.0,12.0,14.0,12.0,1.0,0.0]]\"}"
```